In [ ]:
%reload_ext autoreload
%autoreload 3

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('text', usetex=False)
mpl.rc('font', family='Roboto', size=12)

import numpy as np
from IPython.display import display

import torch
import torchmetrics
import pytorch_lightning as pl
import pandas as pd
from src.cogponder import CogPonderModel
from src.cogponder.data import StroopSRODataset, CogPonderDataModule, NBackSRODataset
from pathlib import Path

from tqdm import tqdm

In [ ]:
CHECKPOINT_PATH = Path('models/checkpoints/stroop/cogponder_epochs-10.ckpt')

In [ ]:
# DEBUG

# DEBUG - Load the checkpoint

model_ckpt = CogPonderModel.load_from_checkpoint(CHECKPOINT_PATH).to('cpu')

print('loading data module...', end='')
dataset = StroopSRODataset(n_subjects=-1, step_duration=10)
datamodule = CogPonderDataModule(dataset, num_workers=8)
datamodule.prepare_data()
print('Done!')

subject_ids, trial_ids, contexts, stimuli, y_human, rt_human, y_correct = datamodule.test_dataset[:]

with torch.no_grad():
    model_ckpt.eval()

    rt_preds = []
    for i in tqdm(range(10)):
        y_pred, ctxt_preds, y_steps, p_halts, rt_pred = model_ckpt(stimuli, subject_ids)
        rt_preds.append(rt_pred.numpy())

    rt_preds = np.array(rt_preds)


In [ ]:
human_stats = pd.DataFrame({
    'condition': dataset.inverse_transform_context(contexts),
    'subject': subject_ids,
    'response_time': rt_human * dataset.step_duration,
    'accuracy': (y_human == y_correct).float() * 100,
    'agent': 'Human data',
})

cogpo_stats = pd.DataFrame({
    'condition': dataset.inverse_transform_context(contexts),
    'subject': subject_ids,
    'response_time': (rt_pred * dataset.step_duration).float(),
    'accuracy': (y_pred == y_correct).float() * 100,
    'agent': 'CogPonder agent',
})

stats = pd.concat([human_stats, cogpo_stats]).reset_index()

# mean_accuracy = stats.groupby(['agent', 'condition'])['accuracy'].mean()
# median_rt = stats.groupby(['agent', 'condition'])['response_time'].mean()
# median_rt, mean_accuracy

plot_data = stats.melt(
    id_vars=['condition', 'agent'],
    value_vars=['response_time', 'accuracy'],
    var_name='measure'
)

In [ ]:

g = sns.catplot(
    data=plot_data, x='condition', y='value', col='measure', hue='agent',
    kind="bar", height=3, aspect=1.2,
    width=.3, errorbar=('ci', 95), capsize=.065, errwidth=1,
    palette='Set2',
    sharex=False, sharey=False,
    estimator='mean',
    col_order=['accuracy', 'response_time'],
    hue_order=['Human data', 'CogPonder agent'],
    order=['congruent', 'incongruent'],
    legend=False
)

g.add_legend(loc='upper left', bbox_to_anchor=(.95, .8))

g.axes[0,0].set_ylabel('% correct')
g.axes[0,0].set_title('Accuracy')
g.axes[0,1].set_ylabel('seconds')
g.axes[0,1].set_title('Response Time (s)')
g.axes[0,0].set_xlabel(None)
g.axes[0,1].set_xlabel(None)


# plt.suptitle('Comparing human and CogPonder performance in the Stroop task', y=1.1)
plt.savefig('outputs/figures/figure3-stroop.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
human_performance = pd.DataFrame({
    'condition': contexts,
    'response_time': rt_human * dataset.step_duration,
    'accuracy': (y_human == y_correct).float(),
    'agent': 'Human data',
})

cogponder_performance = pd.DataFrame({
    'condition': contexts,
    'response_time': rt_pred * dataset.step_duration,
    'accuracy': (y_pred == y_correct).float(),
    'agent': 'CogPonder agent'
})

plot_data = pd.concat([human_performance, cogponder_performance]).reset_index()

plot_data['condition'] = plot_data['condition'].map({0: 'Incongruent', 1: 'Congruent'})

plot_data = plot_data.melt(id_vars=['condition', 'agent'],
               value_vars=['response_time', 'accuracy'],
               var_name='measure'
               )

In [ ]:
g = sns.catplot(
    data=plot_data, x='condition', y='value', col='measure', hue='agent',
    kind="bar", height=3, aspect=1.2,
    width=.3, errorbar=('ci', 95), capsize=.065, errwidth=1,
    palette='Set2',
    sharex=False, sharey=False,
    estimator='mean',
    col_order=['accuracy', 'response_time'],
    hue_order=['Human data', 'CogPonder agent'],
    order=['Congruent', 'Incongruent'],
    legend=False
)

g.add_legend(loc='upper left', bbox_to_anchor=(.95, .8))

g.axes[0,0].set_ylabel('% correct')
g.axes[0,0].set_title('Accuracy')
g.axes[0,1].set_ylabel('seconds')
g.axes[0,1].set_title('Response Time (s)')
g.axes[0,0].set_xlabel(None)
g.axes[0,1].set_xlabel(None)


plt.suptitle('Comparing human and CogPonder performance in the Stroop task', y=1)
plt.tight_layout()
plt.savefig('outputs/figures/figure3-stroop.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# Figure 4

plot_data = stats[['condition', 'agent', 'response_time']].query('response_time < 2000')

g = sns.displot(
    data=plot_data, x='response_time', hue='condition', col='agent',
    kind='hist', kde=True, fill=True, common_norm=True,
    lw=0, alpha=.5, height=3, aspect=1.5,
    kde_kws={'cut': 0, 'clip': (0, 2000)},
    line_kws={'linewidth': 2},
    palette=sns.color_palette(['deepskyblue', 'orangered'], 2),
)

g.set(xlabel='Response time (ms)', ylabel='trials')
g.legend.set_title(None)
g.legend.set_bbox_to_anchor((.45, .5))

# plt.suptitle('Comparing human (N=1) and CogPonder response time distributions in the Stroop task')
# plt.tight_layout()
plt.savefig('outputs/figures/figure4-stroop.png', bbox_inches='tight', dpi=300)

In [ ]:

# sns.relplot(
#     data=plot_data, x='response_time', y='response_time', hue='condition',
#     style='agent',
# )


def boot_ci(x):

    res = []
    for i in range(1000):
        sample = x.sample(1000)
        mean = sample.mean().item()
        medi = sample.median().item()
        res.append([mean, medi])
    return res
    # return torchmetrics.functional.bootstrapped_ci(x, alpha=.05, num_samples=1000)

xx = plot_data.groupby(['condition', 'agent'])['response_time'].apply(boot_ci).apply(pd.Series)
xx = xx.reset_index().melt(id_vars=['condition', 'agent'])
xx[['mean','median']] = xx['value'].apply(pd.Series)

sns.scatterplot(data=xx, x='median', y='mean', hue='condition', style='agent')
# sns.histplot(data=xx, x='mean', y='median', bins=50, pthresh=.1, cmap="mako")
# sns.kdeplot(data=xx, x='mean', y='median', hue='condition', levels=2, linewidths=1)